## Imports

In [38]:
%pip install SemanticScholar
%pip install plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [39]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from semanticscholar import SemanticScholar
import time
import plotly.graph_objects as go

sch = SemanticScholar()

In [40]:
paper = sch.get_paper('10.1519/JSC.0000000000003952')

linked_citations = pd.DataFrame([
    {'title': row['title'],
     'source_DOI': paper.externalIds['DOI'],
     'DOI': row.externalIds['DOI'],
     'citationCount': row['citationCount']}
    for row in paper.citations
])

print(paper['citationCount'])
print(len(paper.citations))
print(paper.referenceCount)


print(linked_citations.head())


23
23
27
                                               title  \
0  Physical Activity and Exercise Intensity Termi...   
1  Beyond the Aesthetics: A Narrative Review of I...   
2  Digital education and exercise therapy versus ...   
3  Resistance training with linear periodization ...   
4  Effects of repeated use of post-exercise infra...   

                     source_DOI                           DOI  citationCount  
0  10.1519/JSC.0000000000003952  10.1249/MSS.0000000000003795              0  
1  10.1519/JSC.0000000000003952  10.1519/ssc.0000000000000924              0  
2  10.1519/JSC.0000000000003952    10.1186/s13063-025-08896-6              0  
3  10.1519/JSC.0000000000003952    10.1016/j.ptsp.2025.05.009              1  
4  10.1519/JSC.0000000000003952    10.3389/fspor.2025.1462901              2  


In [ ]:
# Only using RIR and repetition specific keywords to help with the search
# since we are only interested in the RIR and repetition literature

relevant_keywords = [
    'repetitions in reserve', 'repetition in reserve', 'RIR', 
    'repetition', 'repetitions',
]

# Function to check if a paper is relevant based on keywords
def is_relevant_paper(title):
    lower_title = title.lower()
    return any(keyword.lower() in lower_title for keyword in relevant_keywords)

In [ ]:
# Function to expand the citation network
def expand_citation_network(sch, start_doi, depth, delay):
    G = nx.DiGraph()
    visited = set()

    # Recursively explores papers in both directions (citing and cited)
    # until the depth limit is reached
    def explore(doi, current_depth):
        if current_depth > depth or doi in visited:
            return
        visited.add(doi)

        try:
            paper = sch.get_paper(doi)
        except Exception as e:
            print(f"Error fetching {doi}: {e}")
            return

        # Add current paper (only if relevant)
        citation_count = getattr(paper, "citationCount", 0)
        title = getattr(paper, "title", "Unknown Title")
        
        if is_relevant_paper(title):
            print(f"Adding node: {doi} - {title} ({citation_count} citations)")
            G.add_node(doi, title=title, citationCount=citation_count)
        else:
            print(f"Skipping irrelevant paper: {title}")
            return

        # Explore papers this one cites (successors)
        for cited in getattr(paper, "citations", []):
            try:
                target_doi = (getattr(cited, "externalIds", {}) or {}).get("DOI")
                target_title = getattr(cited, "title", "Unknown")
                if not target_doi:
                    continue
                
                # Only add if relevant
                if is_relevant_paper(target_title):
                    G.add_node(
                        target_doi,
                        title=target_title,
                        citationCount=getattr(cited, "citationCount", 0)
                    )
                    print(f"Adding edge from {doi} to {target_doi}")
                    G.add_edge(doi, target_doi)  # direction: this paper → cites target
            except Exception as e:
                print(f"Error processing citation for {doi}: {e}")
                continue

        # Add references (papers that cite this one)
        for ref in getattr(paper, "references", []):
            try:
                ref_doi = (getattr(ref, "externalIds", {}) or {}).get("DOI")
                ref_title = getattr(ref, "title", "Unknown")
                if not ref_doi:
                    continue
                
                # Only add if relevant
                if is_relevant_paper(ref_title):
                    G.add_node(
                        ref_doi,
                        title=ref_title,
                        citationCount=getattr(ref, "citationCount", 0)
                    )
                    G.add_edge(ref_doi, doi)  # direction: reference → this paper
            except Exception as e:
                print(f"Error processing reference for {doi}: {e}")
                continue

        # Recursive expansion in both directions
        # Explore papers this one cites (successors)
        for next_node in list(G.successors(doi)):
            explore(next_node, current_depth + 1)
            time.sleep(delay)
        
        # Explore papers that cite this one (predecessors)
        for next_node in list(G.predecessors(doi)):
            explore(next_node, current_depth + 1)
            time.sleep(delay)

    explore(start_doi, 0)
    return G


In [47]:
# Example: Start from your Repetitions in Reserve paper
rir_doi = "10.1519/JSC.0000000000003952"
G = expand_citation_network(sch, rir_doi, depth=3, delay=1.5)

Adding node: 10.1519/JSC.0000000000003952 - Repetitions in Reserve Is a Reliable Tool for Prescribing Resistance Training Load (23 citations)
Adding edge from 10.1519/JSC.0000000000003952 to 10.1519/SSC.0000000000000876
Adding edge from 10.1519/JSC.0000000000003952 to 10.1519/JSC.0000000000004805
Adding edge from 10.1519/JSC.0000000000003952 to 10.1177/00315125241241785
Adding node: 10.1519/SSC.0000000000000876 - Repetitions in Reserve: An Emerging Method for Strength Exercise Prescription in Special Populations (3 citations)
Adding edge from 10.1519/SSC.0000000000000876 to 10.7717/peerj.19797
Adding node: 10.7717/peerj.19797 - Exercise type, training load, velocity loss threshold, and sets affect the relationship between lifting velocity and perceived repetitions in reserve in strength-trained individuals (0 citations)
Adding node: 10.1177/00315125241241785 - Feasibility and Usefulness of Repetitions-In-Reserve Scales for Selecting Exercise Intensity: A Scoping Review (9 citations)
Ad

In [48]:
# Compute layout
pos = nx.spring_layout(G, k=0.3, seed=42)

# Create directional edge annotations (arrows)
# Reversed: arrow points FROM source TO citing paper
edge_annotations = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    
    # Swap x0,y0 and x1,y1 to reverse arrow direction
    edge_annotations.append(
        dict(
            x=x0,  # arrow points to x0 (source)
            y=y0,  # arrow points to y0 (source)
            ax=x1,  # arrow comes from x1 (citing paper)
            ay=y1,  # arrow comes from y1 (citing paper)
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            arrowcolor='rgba(100,100,100,0.5)',
            standoff=8
        )
    )

# Extract edge coordinates for lines
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='rgba(100,100,100,0.4)'),
    hoverinfo='none',
    mode='lines'
)

# Extract node coordinates and info
node_x = []
node_y = []
node_size = []
node_color = []
hover_text = []

def wrap_title(title, max_length=50):
    """Wrap title text with line breaks for long titles"""
    if len(title) <= max_length:
        return title
    
    words = title.split()
    lines = []
    current_line = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 <= max_length:
            current_line.append(word)
            current_length += len(word) + 1
        else:
            if current_line:
                lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word)
    
    if current_line:
        lines.append(' '.join(current_line))
    
    return '<br>'.join(lines)

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    citation_count = G.nodes[node].get('citationCount', 0)
    title = G.nodes[node].get('title', 'Unknown Paper')
    
    # Wrap long titles
    wrapped_title = wrap_title(title, max_length=50)
    
    node_size.append(20)  # fixed node size
    node_color.append(citation_count)  # color by citation count
    hover_text.append(f"<b>{wrapped_title}</b><br>Citations: {citation_count}")

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=hover_text,
    marker=dict(
        showscale=True,
        colorscale='Plasma',
        color=node_color,
        size=node_size,
        colorbar=dict(
            thickness=15,
            title=dict(text='Citation Count')
        ),
        line=dict(width=1, color='white')
    )
)

fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title=dict(
            text="Recursive Citation Network for Repetitions in Reserve Literature",
            font=dict(size=20, color='black')
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='white',
        paper_bgcolor='white',
        annotations=edge_annotations
    )
)

fig.show()


In [51]:
#This function will show the 10 most cited papers in the network
def show_most_cited_papers(G):
    # Get all nodes sorted by citation count
    sorted_nodes = sorted(G.nodes(data='citationCount'), key=lambda x: x[1], reverse=True)
    
    # Get top 10 nodes
    top_10 = sorted_nodes[:10]
    return top_10
 
top_citations = show_most_cited_papers(G)

for citation in top_citations:
    print(citation)


('10.1007/s00421-002-0681-6', 1090)
('10.1519/R-18195.1', 335)
('10.1519/JSC.0000000000001049', 309)
('10.1055/s-2005-872825', 219)
('10.1519/SSC.0000000000000218', 187)
('10.5604/20831862.1099047', 121)
('10.1016/j.jshs.2021.01.007', 120)
('10.2478/v10078-008-0008-8', 116)
('10.1519/R-13893.1', 108)
('10.1519/JSC.0000000000002881', 101)
